In [279]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

import random
import math

Загружаем датасет

In [278]:
df = pd.read_csv('src/boston.csv')

Преобразуем данные в массив numpy

In [267]:
X_np = df.drop(df.columns[-1], axis=1).to_numpy()
y_np = df['MEDV'].to_numpy()


Преобразуем данные в обычные массивы

In [273]:
X = X_np.tolist()
y = y_np.tolist()

In [301]:
def train_test_split_np(X, y, test_size=0.2, random_state=None):
    if random_state is not None:
        np.random.seed(random_state)
        
    # Проверка входных данных
    if X.shape[0] != y.shape[0]:
        raise ValueError("Количество строк в X и y должно совпадать.")
    if not 0 < test_size < 1:
        raise ValueError("test_size должен быть между 0 и 1.")

    # Получаем количество образцов
    n_samples = X.shape[0]
    
    # Вычисляем количество тестовых образцов
    n_test_samples = int(n_samples * test_size)
    
    # Создаем случайную перестановку индексов
    indices = np.random.permutation(n_samples)
    
    # Разделяем индексы
    test_indices = indices[:n_test_samples]
    train_indices = indices[n_test_samples:]
    
    # Разделяем данные
    X_train, X_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]
    
    return X_train, X_test, y_train, y_test

In [ ]:
def train_test_split(X: list, y: list, test_size: float = 0.2, random_state: int = None) -> tuple:
    """
    Разделяет данные на обучающую и тестовую выборки.
    
    :param X: Список признаков или входных данных
    :param y: Список целевых значений или меток
    :param test_size: Доля данных для тестовой выборки (от 0 до 1), по умолчанию 0.2
    :param random_state: Управляет воспроизводимостью результатов, по умолчанию None
    :return: Кортеж из четырех списков (X_train, X_test, y_train, y_test)
    :raises ValueError: Если количество элементов в X и y не совпадает
                       или если test_size не находится в диапазоне (0, 1)
    """
    if not random_state is None:
        random.seed(random_state)

    # Проверка входных данных
    if len(X) != len(y):
        raise ValueError('Количество элементов X и y должно совпадать')
    if not 0 < test_size < 1:
        raise ValueError('test_size должен быть между 0 и 1')

    n_samples = len(X)
    n_test_samples = int(n_samples * test_size)

    # Случайная перестановка индексов
    indexes = list(range(n_samples))
    random.shuffle(indexes)

    test_indexes = indexes[:n_test_samples]
    train_indexes = indexes[n_test_samples:]

    # Разделение данных
    X_train = [X[i] for i in train_indexes]
    X_test = [X[i] for i in test_indexes]
    y_trian = [y[i] for i in train_indexes]
    y_test = [y[i] for i in test_indexes]

    return X_train, X_test, y_trian, y_test    

Разделяем данные на тренировочные и тестовые

In [277]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [303]:
def linear_regression_np(X, y):

    # 1. Подготовка данных: нормализация признаков
    X_mean = np.mean(X, axis=0)
    X_std = np.std(X, axis=0)
    # Избегаем деления на ноль
    X_std[X_std == 0] = 1
    X_normalized = (X - X_mean) / X_std
    
    # Добавление столбца единиц для свободного члена (intercept)
    m, n = X_normalized.shape
    X_b = np.column_stack([np.ones(m), X_normalized])

    learning_rate = 0.001
    weights = np.random.randn(n + 1) * 0.01

    for i in range(50000):
        y_pred = X_b @ weights
        error = y_pred - y

        gradient = (2 / m) * X_b.T @ error
        weights = weights - learning_rate * gradient

    return weights, X_mean, X_std

def predict_np(X, weights, X_mean, X_std):
    X_normalized = (X - X_mean) / X_std
    m = X_normalized.shape[0]
    X_b = np.column_stack([np.ones(m), X_normalized])
    return X_b @ weights


In [280]:
def linear_regression(X: list, y: list, learning_rate: float = 0.001, tolerance: float = 1e-7) -> tuple:
    """
    Линейная регрессия методом градиентного спуска.
    
    :param X: Список списков признаков (двумерный массив)
    :param y: Список целевых значений (одномерный массив)
    :param learning_rate: Скорость обучения для градиентного спуска, по умолчанию 0.001
    :param tolerance: Порог сходимости для остановки обучения (минимальное изменение ошибки),
        по умолчанию 1e-7
    :return: Кортеж из трех элементов (weights, X_mean, X_std)
    """
    
    # Нормализация признаков
    # Среднее значение по каждому признаку
    n_features = len(X[0])
    X_mean = []
    for i in range(n_features):
        sum_val = sum(row[i] for row in X)
        X_mean.append(sum_val / len(X))

    # Стандартное отклонение по каждому признаку
    X_std = []
    for i in range(n_features):
        sum_squared_diff = sum((row[i] - X_mean[i]) ** 2 for row in X)
        std_val = math.sqrt(sum_squared_diff / len(X))
        # Избегаение деления на ноль
        if std_val == 0:
            std_val = 1
        X_std.append(std_val)

    # Нормализация признаков
    X_normalized = []
    for i in range(len(X)):
        normalized_row = []
        for j in range(n_features):
            normalized_value = (X[i][j] - X_mean[j] / X_std[j])
            normalized_row.append(normalized_value)
        X_normalized.append(normalized_row)

    # Добавление столбца единиц для свободного члена (intercept)
    m = len(X_normalized)
    n = len(X_normalized[0])
    X_b = []
    for i in range(m):
        X_normalized[i].append(1.0)
        X_b.append(X_normalized[i])
    
    # Инициализация весов
    weights = []
    for i in range(n + 1):
        # Генерирация случайных весов (в диапазоне [-0.01, 0.01])
        random_weight = (random.random() - 0.5) * 0.2
        weights.append(random_weight)

    # Градиентный спуск
    prev_error = float('inf')
    iteration = 0
    max_iteration = 100000
    while iteration < max_iteration:
        # Вычисление предиктов
        y_pred = []
        for i in range(m):
            pred = 0
            for j in range(n + 1):
                pred += X_b[i][j] * weights[j]
            y_pred.append(pred)

        # Вычисление ошибок
        error = []
        for i in range(m):
            error.append(y_pred[i] - y[i])

        # Вычисление среднеквадратичной ошибки
        mse = sum(e ** 2 for e in error) / m

        if abs(prev_error - mse) < tolerance:
            break

        prev_error = mse

        # Вычисление градиента
        gradient = []
        for i in range(n + 1):
            grad_sum = 0
            for j in range(m):
                grad_sum += X_b[j][i] * error[j]
            gradient.append((2 / m) * grad_sum)

        # Обновление весов
        for i in range(n + 1):
            weights[i] = weights[i] - learning_rate * gradient[i]

        iteration += 1

    return weights, X_mean, X_std

def predict(X: list, weights: list, X_mean: list, X_std: list) -> tuple:
    """
    Делает предсказания на основе обученной модели линейной регрессии.
    
    :param X: Список списков признаков для предсказания
    :param weights: Веса модели, полученные из linear_regression
    :param X_mean: Средние значения признаков, использованные при обучении
    :param X_std: Стандартные отклонения признаков, использованные при обучении
    :return: Список предсказанных значений
    """

    # Нормализция признаков
    X_normalized = []
    n_features = len(X[0]) if X else 0
    for i in range(len(X)):
        normalized_row = []
        for j in range(n_features):
            normalized_value = (X[i][j] - X_mean[j]) / X_std[j]
            normalized_row.append(normalized_value)
        X_normalized.append(normalized_row)
    
    # Добавление столбца единиц для свободного члена
    m = len(X_normalized)
    X_b = []
    for i in range(m):
        row = [1.0]  # столбец единиц
        row.extend(X_normalized[i])
        X_b.append(row)
    
    # Вычисление предсказаний
    predictions = []
    for i in range(m):
        pred = 0
        for j in range(len(weights)):
            pred += X_b[i][j] * weights[j]
        predictions.append(pred)
    
    return predictions

In [302]:
X_train, X_test, y_train, y_test = train_test_split_np(X_np, y_np)

In [304]:
ww, xm, xs =linear_regression_np(X_train, y_train)

In [305]:
a = predict(X_test, ww, xm, xs)
for i in range(len(a)):
    print(a[i], '\t\t', y_test[i], '\t\t', b[i])

8.848447827479163 		 11.9 		 17.29558059208722
20.05356278048241 		 20.3 		 12.14881480303195
37.481017779479146 		 44.0 		 22.481784461712497
18.46883282968789 		 20.8 		 22.202188867535586
17.94147498071456 		 19.3 		 37.16328631468496
33.44899829816234 		 28.5 		 24.915254636052772
34.553649294116425 		 35.4 		 19.455161955636836
32.28074905358561 		 31.1 		 37.511092390231866
22.16341604791499 		 17.8 		 24.429823483247873
12.840653950377689 		 10.5 		 20.17208927329338
22.5826612503387 		 25.0 		 14.374785228367998
15.133134512320009 		 14.9 		 19.361639540292664
23.140554787099454 		 20.4 		 32.25102801323425
24.393009694453514 		 20.5 		 32.708276658353974
21.504963495779116 		 21.0 		 22.869524474348186
28.133910167261035 		 26.6 		 20.232744140187492
13.221440761872563 		 7.5 		 17.18668530858755
18.749205326883953 		 18.0 		 26.402739552736584
30.534198435359954 		 29.1 		 14.626176241815624
36.27393614383037 		 33.3 		 14.207647351941354
17.301194070519216 		 19.1 		 20.5275

In [287]:
w, x_m, x_std = linear_regression(X_train, y_train, learning_rate=0.0001)

In [299]:
a = predict(X_test, w, x_m, x_std)
for i in range(len(a)):
    print(a[i], '\t\t', y_test[i], '\t\t', b[i])

14.573096071784823 		 17.1 		 17.29558059208722
12.822192853185467 		 15.6 		 12.14881480303195
17.05840810124809 		 21.4 		 22.481784461712497
19.433542529620286 		 22.7 		 22.202188867535586
33.40470279263016 		 48.3 		 37.16328631468496
21.75442951996798 		 21.5 		 24.915254636052772
14.763887451466271 		 20.6 		 19.455161955636836
32.926195224825975 		 44.0 		 37.511092390231866
20.999748366744207 		 21.7 		 24.429823483247873
16.534545947134824 		 17.7 		 20.17208927329338
10.510727498657173 		 19.0 		 14.374785228367998
16.91199452970177 		 17.1 		 19.361639540292664
29.011730182148547 		 33.2 		 32.25102801323425
28.31906206216913 		 27.0 		 32.708276658353974
19.559289713658806 		 20.4 		 22.869524474348186
16.58132328410346 		 13.3 		 20.232744140187492
14.178084927461262 		 14.9 		 17.18668530858755
22.62628631096238 		 23.8 		 26.402739552736584
9.19937666760862 		 11.7 		 14.626176241815624
10.065534272121624 		 11.5 		 14.207647351941354
17.95045721678558 		 23.1 		 20.527

In [294]:
model = LinearRegression()
model.fit(X, y)
b = model.predict(X_test)